In [58]:
import json
import os
import pandas as pd
from openai import OpenAI

openai_api_key = os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)
def get_openai_vision_response(prompt_text: str, img_url: str):
    response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt_text},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": img_url,
                        },
                    },
                ],
            }
        ],
        max_tokens=5,
    )
    
    return response.choices[0].message.content
    
def load_input_data():
    with open('../input.json', 'r') as file:
        json_data = json.load(file)
    return json_data

def color_cells(row):
    if row["In-Stock"] == "Yes" and row["Out-of-Stock"] == "No":
        return ["background-color: #296644"] * len(row)
    else:
        return ["background-color: #82323a"] * len(row)

In [59]:
data = load_input_data()

prompt = "Is the product available for purchase? Please answer 'Yes' or 'No' only."
df = pd.DataFrame(columns=["Source", "In-Stock", "Out-of-Stock"])

for i in range(0, len(data)):
    in_stock_img_url = data[i]['in_stock']
    in_stock_response = get_openai_vision_response(prompt, in_stock_img_url)

    out_of_stock_img_url = data[i]['out_of_stock']
    out_of_stock_response = get_openai_vision_response(prompt, out_of_stock_img_url)

    df.loc[i] = [data[i]["name"], in_stock_response, out_of_stock_response]

print(f"{prompt}")
df.style.apply(color_cells, axis=1)

Is the product available for purchase? Please answer 'Yes' or 'No' only.


,Source,In-Stock,Out-of-Stock
0,alkosto.com,Yes,No
1,alza.cz,Yes,No
2,bestbuy.com,Yes,No
3,biccamera.com,Yes,No
4,coolblue.nl,Yes,No
5,costco.com,Yes,No
6,digitec.ch.de,Yes,No
7,yodobashi.com,Yes,No
8,yamada.denkiweb.com,Yes,No
9,walmart.com,Yes,No
